In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
import matplotlib.pyplot as plt

In [ ]:
#Load Pre-Processed Train / Test Data

train_df = pd.read_csv('Train-Data.csv')
test_df = pd.read_csv('Test-Data.csv')

print(f'Train Length: {len(train_df)}, Test Length: {len(test_df)}')

In [ ]:
#Prepare Data For ML input
trainx = np.array(train_df.iloc[:,:-1].values.tolist())
trainy = np.array(train_df['label'].tolist())

testx = np.array(test_df.iloc[:,:-1].values.tolist())
testy = np.array(test_df['label'].tolist())

In [ ]:
#Validate Correct Input Shapes
print(f'Trainx: {trainx.shape}, Trainy: {trainy.shape}')
print(f'Testx: {testx.shape}, Testy: {testy.shape}')

In [ ]:
#Encode Classification Labels
le = LabelEncoder()

le.fit(trainy)

trainy = le.transform(trainy)
testy = le.transform(testy)

In [ ]:
#Build Fully Connected Neural Network Model
FCNN = Sequential()
FCNN.add(Dense(32, input_dim=7, activation='relu'))
FCNN.add(Dense(64, activation='relu'))
FCNN.add(Dense(22, activation='softmax'))
FCNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Display Summary of Model Architecture
FCNN.summary()

In [ ]:
#Train the fully connected neural network
FCNN.fit(trainx, trainy, epochs=50, batch_size=32)

In [ ]:
#Make predictions
FCNN_Preds = FCNN.predict(testx)
FCNN_Preds = np.array([np.argmax(i) for i in FCNN_Preds])

In [ ]:
#Performance metrics functin
def get_performance_metrics(preds):
    metrics = {}
    metrics['acc'] = accuracy_score(testy, preds)
    metrics['prec'] = precision_score(testy, preds, average='macro')
    metrics['rec'] = recall_score(testy, preds, average='macro')
    metrics['f1'] = f1_score(testy, preds, average='macro')
    metrics['cm'] = confusion_matrix(testy, preds)
    return metrics

In [ ]:
#Calculate metrics
FCNN_Metrics = get_performance_metrics(FCNN_Preds)

In [ ]:
#Build CNN
CNN = Sequential()
CNN.add(Conv1D(filters=32, kernel_size=3, input_shape=(7, 1), activation='relu'))
CNN.add(Flatten())
CNN.add(Dense(128, activation='relu'))
CNN.add(Dense(22, activation='softmax'))
CNN.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#Reshape the x arrays to adapt to the input requirements of CNN. Train the CNN.
trainx_reshaped = trainx.reshape(trainx.shape[0], 7, 1)
testx_reshaped = testx.reshape(testx.shape[0], 7, 1)
CNN.fit(trainx_reshaped, trainy, epochs=50, batch_size=32)

In [ ]:
#Make CNN Predictions
CNN_Preds = CNN.predict(testx)
CNN_Preds = np.array([np.argmax(i) for i in CNN_Preds])

In [ ]:
#Calculate CNN Metrics
CNN_Metrics = get_performance_metrics(CNN_Preds)

In [ ]:
#Display both sets of metrics
print("FCNN Metrics:", {k:round(v, 3) for k,v in FCNN_Metrics.items() if k != 'cm'})
print("CNN Metrics:", {k:round(v, 3) for k,v in CNN_Metrics.items() if k != 'cm'})

In [ ]:
#Visualise Metrics
FCNN_Dict = {k:round(v, 3) for k,v in FCNN_Metrics.items() if k != 'cm'}
CNN_Dict = {k:round(v, 3) for k,v in CNN_Metrics.items() if k != 'cm'}

labels = list(FCNN_Dict.keys())
FCNN_vals = list(FCNN_Dict.values())
CNN_vals = list(CNN_Dict.values())

x = np.arange(len(labels))
width = 0.3

fig, ax = plt.subplots()

rects1 = ax.bar(x - width, FCNN_vals, width, label='FCNN', alpha=0.7)
rects2 = ax.bar(x, CNN_vals, width, label='CNN', alpha=0.7)

ax.set_ylim(0.9, 1.0)
ax.set_xlabel('Metrics')
ax.set_ylabel('Value')
ax.set_title('Values by metric and model')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
ax.legend(loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
#Visualise Confuusion Matricess
fig, axes = plt.subplots(1, 2, figsize=(20, 5))

cms = [FCNN_Metrics['cm'], CNN_Metrics['cm']]
titles = ['FCNN', 'CNN']

for ax, cm, title in zip(axes, cms, titles):

    cax = ax.matshow(cm, cmap=plt.cm.Blues)
    plt.colorbar(cax, ax=ax)
    ax.set_title(title, pad=20)
    ax.set_xticks(np.arange(len(le.classes_)))
    ax.set_yticks(np.arange(len(le.classes_)))
    ax.set_xticklabels(le.classes_, rotation=90)
    ax.set_yticklabels(le.classes_)
    
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, str(cm[i, j]), ha='center', va='center', color='red')

plt.tight_layout()
plt.show()